In [12]:
import datetime
from alpaca_trade_api.stream import Stream
from alpaca_trade_api.rest import REST, TimeFrame
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
from trading_strategy import TradingStrategy
from data_preparation import load_data, preprocess_and_save_data, create_sequences, preprocess_data
import yaml
import pickle
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import requests
from datetime import datetime, timedelta
import time

In [3]:
import alpaca_trade_api as tradeapi

API_KEY = "PK6RB7JZX8K8UG3IZGQ0"
API_SECRET = "J9R6zlh3olUA599LeYQ1rSNn7Fs4SNUvl1b3fxog"
APCA_API_BASE_URL = "https://paper-api.alpaca.markets"

# api = tradeapi.REST(API_KEY, API_SECRET, APCA_API_BASE_URL, 'v2')
alpaca = REST(API_KEY, API_SECRET, base_url='https://paper-api.alpaca.markets')
load_dotenv()
api_key = os.getenv('ALPHA_VANTAGE_API_KEY')

In [4]:
with open('settings.yml', 'r') as f:
    dat = yaml.load(f, Loader=yaml.SafeLoader)

In [15]:
# Function to get the current position for a symbol
def get_current_position(symbol):
    try:
        position = alpaca.get_position(symbol)
        return float(position.qty)
    except:
        return 0

# Function to place a trade using Alpaca API
def place_trade(symbol, side, qty):
    alpaca.submit_order(
        symbol=symbol,
        qty=qty,
        side=side,
        type='market',
        time_in_force='gtc'
    )

# Function to check if it's 10 minutes before market close
def is_market_closing_soon():
    clock = alpaca.get_clock()
    market_close_time = clock.next_close.replace(tzinfo=None)
    current_time = datetime.utcnow()
    return current_time >= (market_close_time - timedelta(minutes=10))

def fetch_realtime_data(symbol):
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval=1min&apikey={api_key}&outputsize=compact"
    response = requests.get(url)
    data = response.json()
    
    if 'Time Series (1min)' in data:
        df = pd.DataFrame.from_dict(data['Time Series (1min)'], orient='index', dtype='float')
        df = df.rename(columns={"1. open": "Open", "2. high": "High", "3. low": "Low", "4. close": "Close", "5. volume": "Volume"})
        df.index = pd.to_datetime(df.index)
        df = df.sort_index()
        return df
    else:
        raise ValueError("Error fetching data from Alpha Vantage")

# Function to close all positions
def close_all_positions():
    positions = alpaca.list_positions()
    for position in positions:
        side = 'sell' if float(position.qty) > 0 else 'buy'
        alpaca.submit_order(
            symbol=position.symbol,
            qty=abs(int(position.qty)),
            side=side,
            type='market',
            time_in_force='gtc'
        )

In [9]:
cur_pos = {stock:0 for stock in dat['stocks']}
model_path = '/Users/aditya/psr_v1/lstm_model_v3.keras'
model = load_model(model_path)

2024-06-07 01:19:17.450574: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-07 01:19:17.451757: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-07 01:19:17.452628: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [16]:
while True:
        if is_market_closing_soon():
                print("Market is closing soon, closing all positions.")
                close_all_positions()
                break
        for stock in dat['stocks']:
                data = fetch_realtime_data(stock)
                with open(f'{stock}_scaler.pkl', 'rb') as file:
                        scaler = pickle.load(file)
                processed_data = preprocess_and_save_data(data,stock)
                # Now create the sequences from the processed data
                sequences = create_sequences(processed_data.values, dat['training_duration'], dat['num_features'])
                yhat = model.predict(sequences[-1::])
                if ((scaler.inverse_transform(yhat)[0][3] - data.iloc[-1,3])/data.iloc[-1,3])>dat['profit_percent']:
                        if cur_pos[stock]<=0:
                                print("Placing a buy order.")
                                place_trade(stock, 'buy', 1)
                        else:
                                print("Already in a long position, no buy order placed.")
                elif ((scaler.inverse_transform(yhat)[0][3] - data.iloc[-1,3])/data.iloc[-1,3])<-dat['profit_percent']:
                        if cur_pos[stock] >= 0:  # Only sell if flat or long
                                print("Placing a sell order (shorting).")
                                place_trade(stock, 'sell', 1)
                        else:
                                print("Already in a short position, no sell order placed.")
        
        time.sleep(60)
        

Market is closing soon, closing all positions.
